In [29]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lesak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
def find_replacements(df):
    sia = SentimentIntensityAnalyzer()
    replacements = {}

    for index, row in df.iterrows():
        reference_sentence = row['reference']
        translation_sentence = row['translation']
        
        reference_tokens = word_tokenize(reference_sentence)
        translation_tokens = word_tokenize(translation_sentence)

        for ref_token, trn_token in zip(reference_tokens, translation_tokens):
            ref_sentiment = sia.polarity_scores(ref_token)['compound']
            if ref_sentiment < 0:
                replacements[ref_token] = trn_token

    return replacements


In [31]:
# Создайте DataFrame (замените 'data' на ваш фактический DataFrame)
fitrered_df=pd.read_csv("../data/interm/preprocessed_filtered.csv")

# Вызовите функцию и получите словарь замен
replacements = find_replacements(fitrered_df)

# Выведите словарь
print(replacements)


{'disorder': 'behind', 'kill': 'get', 'fuck': 'are', 'Shit': 'what', 'leave': 'your', 'poor': 'poor', 'bastard': ',', 'no': 'have', 'killing': 'himself', 'savage': 'wild', 'contemptuously': 'with', 'waste': 'hell', 'swear': 'family', 'bitch': '.', 'irresponsible': '!', 'die': 'died', 'nuts': 'about', 'dead': 'sure', 'foolhardy': 'pontiff', 'hell': 'hell', 'shit': '.', 'damn': 'you', 'Disgusting': 'A', 'fight': ',', 'Hurt': 'hit', 'Fuck': 'because', 'ass': 'me', 'rob': ',', 'motherfucker': 'man', 'annoying': 'mother', 'arrogant': 'degrading', 'rape': 'and', 'Damn': 'I', 'fucked': 'screwed', 'suicide': 'suicide', 'attack': '!', 'Miss': '.', 'Bullshit': 'nothing', 'loser': 'lost', 'goddamn': 'good', 'freaky': '.', 'hiding': 'hiding', 'hard': 'until', 'stealing': 'in', 'assaulted': 'he', 'lose': 'see', 'bitter': 'bitter', 'impatient': 'of', 'painful': '.', 'crush': 'when', 'stop': 'my', 'Crap': 'hell', 'goddam': 'think', 'gun': 'a', 'scared': 'scared', 'miserable': 'restaurant', 'shoot': '

In [32]:
test_df = pd.read_csv("../data/interm/test.csv")

In [33]:
for original, replacement in replacements.items():
    test_df['translate'] = test_df['reference'].str.replace(original, replacement)

In [37]:
test_df['translate'] = test_df['reference']

# Проходим по словарю замен и заменяем соответствующие слова в столбце "translate"
for original, replacement in replacements.items():
    test_df['translate'] = test_df['translate'].str.replace(original, replacement)

In [38]:
test_df.head()

,reference,ref_tox,translate
0,"They're all laughing at us, so we'll kick your...",0.999492,"They're all lasomethinging at us, so we'll kic..."
1,"Come on, Cal, leave that shit alone.",0.999637,"Come on, Cal, your that . of."
2,I like that shit.,0.999594,I like that ..
3,How is this not porn? This is porn that comes ...,0.933948,How is this havet porn? This is porn that come...
4,"Now, I understand you got your grievances with...",0.999603,".w, I understand you got your ands with these ..."


In [111]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    tokens = [stemmer.stem(word) for word in tokens]
    return " ".join(tokens)

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')


In [113]:
from tensorflow.keras.models import load_model


model = load_model('model.h5')

In [114]:
X_test = test_df['translate']
X_test_preprocessed = [preprocess_text(text) for text in X_test]
X_test_tfidf = tfidf_vectorizer.transform(X_test_preprocessed)

NotFittedError: The TF-IDF vectorizer is not fitted

In [115]:
y_pred = model.predict(X_test_tfidf.toarray())

157/157 [==============================] - 0s 2ms/step


In [116]:
y_pred

array([[0.9356336 ],
       [0.9890278 ],
       [0.2268527 ],
       ...,
       [0.11515912],
       [0.8779996 ],
       [0.02924436]], dtype=float32)

In [117]:
test_df['tr_tox'] = y_pred

In [119]:
test_df['result'] = test_df['ref_tox']-test_df['tr_tox']

In [120]:
test_df['result']

0       0.063859
1       0.010609
2       0.772742
3       0.033310
4       0.907080
          ...   
4995    0.745048
4996    0.984663
4997    0.832143
4998    0.117500
4999    0.965515
Name: result, Length: 5000, dtype: float64